<a href="https://colab.research.google.com/github/suheon927/PerceptronPractice/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

x = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
      [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
y = [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]

weights = [0.0, 0.0, 0.0, 0.0] # 각 차원에 대한 가중치를 설정
bias = 0.0  #바이어스

l_rate = 0.1 # 학습률을 0.1로 증가시킴
n_epoch = 250

def relu(x) : # ReLU 함수
  return max(0, x)

# 뉴런의 출력 계산 함수
def calculate(input):
    global weights
    global bias
    activation1 = bias # 바이어스1
    activation2 = bias # 바이어스2
    for i in range(4): # 입력신호 총합 게산
        activation1 += weights[i] * input[i] # 가중치와 입력 곱의 누적합1
        activation2 += weights[i] * input[i] # 가중치와 입력 곱의 누적합2
    output = relu(activation1)  # ReLU 함수 적용
    output = relu(activation2)
    return 1.0 if output >= 0.5 else 0.0  # 출력 이진화
    return relu(activation) # 시그모이드 함수 적용
    if activation >= 0.0: # 스텝 활성화 함수
        return 1.0
    else:
        return 0.0

# 학습 알고리즘
def train_weights(x, y, l_rate, n_epoch):
    global weights
    global bias
    for epoch in range(n_epoch): # 에포크 반복

        sum_error = 0.0
        print('sum_error', sum_error)
        for row, target in zip(x, y): # 데이터셋을 반복
            actual = calculate(row) # 실제 출력 계산
            error = target - actual # 실제 출력 계산
            bias = bias + l_rate * error
            sum_error += error**2 # 오류의 제곱 계산
            for i in range(4): # 가중치 변경
                weights[i] = weights[i] + l_rate * error * row[i]
            print(weights, bias)
        print('에포크 번호=%d, 학습률=%.3f, 오류=%.3f' % (epoch, l_rate, sum_error))
    return weights

weights = train_weights(x, y, l_rate, n_epoch)
print(weights, bias)

sum_error 0.0
[0.0, 0.0, 0.0, 0.0] 0.0
[0.0, 0.0, 0.0, 0.1] 0.1
[0.0, 0.0, 0.1, 0.1] 0.2
[0.0, 0.0, 0.1, 0.1] 0.2
[0.0, 0.1, 0.1, 0.1] 0.30000000000000004
[0.0, 0.0, 0.1, 0.0] 0.20000000000000004
[0.0, 0.0, 0.1, 0.0] 0.20000000000000004
[0.0, 0.1, 0.2, 0.1] 0.30000000000000004
[0.1, 0.1, 0.2, 0.1] 0.4
[0.0, 0.1, 0.2, 0.0] 0.30000000000000004
[-0.1, 0.1, 0.1, 0.0] 0.20000000000000004
[0.0, 0.1, 0.2, 0.1] 0.30000000000000004
[0.0, 0.1, 0.2, 0.1] 0.30000000000000004
[0.0, 0.1, 0.2, 0.1] 0.30000000000000004
[0.0, 0.1, 0.2, 0.1] 0.30000000000000004
[-0.1, 0.0, 0.1, 0.0] 0.20000000000000004
에포크 번호=0, 학습률=0.100, 오류=10.000
sum_error 0.0
[-0.1, 0.0, 0.1, 0.0] 0.20000000000000004
[-0.1, 0.0, 0.1, 0.1] 0.30000000000000004
[-0.1, 0.0, 0.2, 0.1] 0.4
[-0.1, 0.0, 0.1, 0.0] 0.30000000000000004
[-0.1, 0.1, 0.1, 0.0] 0.4
[-0.1, 0.0, 0.1, -0.1] 0.30000000000000004
[-0.1, 0.0, 0.1, -0.1] 0.30000000000000004
[-0.1, 0.1, 0.2, 0.0] 0.4
[0.0, 0.1, 0.2, 0.0] 0.5
[-0.1, 0.1, 0.2, -0.1] 0.4
[-0.2, 0.1, 0.1, -0.1